In [1]:
from CosinorPy import file_parser, cosinor, cosinor1, cosinor_nonlin
import numpy as np
import pandas as pd
import os

In [2]:
import importlib
importlib.reload(cosinor)
importlib.reload(cosinor1)
importlib.reload(cosinor_nonlin)
importlib.reload(file_parser)

<module 'CosinorPy.file_parser' from 'C:\\Users\\miham\\OneDrive - Univerza v Ljubljani\\Research\\CosinorPy\\code\\CosinorPy\\file_parser.py'>

## Generate test data

In [3]:
df = file_parser.generate_test_data(phase = np.pi, n_components = 1, name="sym", lin_comp = 0, noise=0.3, replicates = 3, noise_simple=0)
df2 = file_parser.generate_test_data(phase = 0, n_components = 1, name="sym_lin_comp", lin_comp = 0.1, noise=0.3, replicates = 3, noise_simple=0)
df = df.append(df2, ignore_index=True)

df2 = file_parser.generate_test_data(phase = np.pi, n_components = 3, name="asym", lin_comp = 0, noise=0.3, replicates = 3, noise_simple=0)
df = df.append(df2, ignore_index=True)
df2 = file_parser.generate_test_data(phase = 0, n_components = 3, name="asym_lin_comp", lin_comp = 0.1, noise=0.3, replicates = 3, noise_simple=0)
df = df.append(df2, ignore_index=True)


df2 = file_parser.generate_test_data(phase = 0, n_components = 1, name="sym_damped", amplification = -0.04, noise=0.3, replicates = 3, time_step=2, noise_simple=0, max_time = 72)
df = df.append(df2, ignore_index=True)
df2 = file_parser.generate_test_data(phase = np.pi, n_components = 1, name="sym_forced", amplification = 0.04, noise=0.3, replicates = 3, time_step=2, noise_simple=0, max_time = 72
df = df.append(df2, ignore_index=True)


df2 = file_parser.generate_test_data(phase = 0, n_components = 3, name="asym_damped", amplification = -0.04, noise=0.3, replicates = 3, time_step=2, noise_simple=0,  max_time = 72)
df = df.append(df2, ignore_index=True)
df2 = file_parser.generate_test_data(phase = np.pi, n_components = 3, name="asym_forced", amplification = 0.04, noise=0.3, replicates = 3, time_step=2, noise_simple=0, max_time = 72)
df = df.append(df2, ignore_index=True)

SyntaxError: invalid syntax (<ipython-input-3-7feb6c0a48a6>, line 14)

Let us plot the data first:

In [ ]:
cosinor.plot_data(df)

In [ ]:
#file_parser.export(df,"test_data\\independent_data_nonlin.xlsx")

In [ ]:
df = file_parser.read_excel("test_data\\independent_data_nonlin.xlsx")

## Cosinor analysis

Identify the best models and/or the best periods (possible periods can be given as an interval or as a single value).

In [ ]:
df_results = cosinor.fit_group(df, n_components = [1,2,3], period=24, plot=False, plot_phase=False, lin_comp=False)

Get the best models (the best fitting periods and the best fitting models - in dependence on the number of components; by default the criterium is p-value)

In [ ]:
df_best_models = cosinor.get_best_models(df, df_results, n_components = [1,2,3])

In [ ]:
df_best_models

... and plot these models

In [ ]:
cosinor.plot_df_models(df, df_best_models, plot_phase=False)#, folder="nonlin_basic_models")

We can analyse these models more in details using bootstrap analysis.

In [ ]:
df_results_extended = cosinor.analyse_best_models(df, df_best_models)

In [ ]:
df_results_extended[['test', 'amplitude', 'q(amplitude)', 'acrophase', 'q(acrophase)']].round(3)

Obviously, some of these fits could be better by introducing linear component and/or amplification coefficient.

## Generalized cosinor1 analysis

First, we will use a single-component model. Using these functions, only 1-component models can be used, but the statistics can be obtained without boostrapping (so this is faster).

In [ ]:
df_results = cosinor_nonlin.fit_generalized_cosinor_group(df, period = 24, plot=True)#, folder="nonlin_gen1_models") 

In [ ]:
df_results[['test', 'amplitude', 'q(amplitude)', 'acrophase', 'q(acrophase)', 'amplification', 'q(amplification)', 'lin_comp', 'q(lin_comp)']].round(3)

## Comparison analysis using generalized cosinor1

The generalised 1-component cosinor model can also be used to compare pairs in a similar way as the basic 1-component model. First we need to define the pairs to compare:

In [ ]:
pairs = [("sym", "sym_lin_comp"),("asym", "asym_lin_comp"), ("sym_damped", "sym_forced"), ("asym_damped", "asym_forced")]

Then, we can run the wrapper function that uses the same model:

In [ ]:
df_results_compare = cosinor_nonlin.fit_generalized_cosinor_compare_pairs_dependent(df, pairs, period=24, plot=True)

In [ ]:
df_results_compare

... or two independent models, which means that each model can be fitted with a different period:

In [ ]:
df_results_compare = cosinor_nonlin.fit_generalized_cosinor_compare_pairs_independent(df, pairs, period1=24, period2=24, plot=True)

In [ ]:
df_results_compare

## Generalized multicomponent cosinor analysis
A better fit would be obtained in some cases (e.g., test7 and test8) if a multicomponent cosinor model would be used. 

We can use a wrapper function that automatically identifies the optimal number of components for each dataset, and plots and stores the results in a data frame.

In [ ]:
df_best_models = cosinor_nonlin.fit_generalized_cosinor_n_comp_group_best(df, period=24, n_components = [1,2,3], plot=True)#, folder="nonlin_gen_models") 

In [ ]:
df_best_models[['test', 'n_components']]

In [ ]:
df_best_models

Alternatively, we can also fix the number of components to the same value for all datasets:

In [ ]:
df_3comp_models = cosinor_nonlin.fit_generalized_cosinor_n_comp_group(df, period=24, n_components = 3, plot=True)

In [ ]:
df_3comp_models

However, the significance of amplitudes and acrophases being different than zero now needs to be evaluated using bootstrap.

We can do this using the best models for each dataset:

In [ ]:
df_bootstrap = cosinor_nonlin.bootstrap_generalized_cosinor_n_comp_group_best(df, df_best_models, bootstrap_size=100)

In [ ]:
df_bootstrap[['test', 'n_components', 'amplitude', 'q(amplitude)', 'acrophase', 'q(acrophase)', 'amplification', 'q(amplification)', 'lin_comp', 'q(lin_comp)']].round(3)

Alternatively, we can also fix the number of components to the same value for all the datasets:

In [ ]:
df_bootstrap3 = cosinor_nonlin.bootstrap_generalized_cosinor_n_comp_group(df, period=24, n_components=3, bootstrap_size=100)

In [ ]:
df_bootstrap3

## Comparison using generalized multicomponent cosinor analysis
This analysis relies on the bootsrapping as in the basic multicomponent cosinor analysis. However, it is not necessary to run bootstrap again, since we can use the results produced in previous steps. Namely, we will use the confidence intervals of the basic bootstrap analyses.

First, we need to define the pairs to compare:

In [ ]:
pairs = [("sym", "sym_lin_comp"),("asym", "asym_lin_comp"), ("sym_damped", "sym_forced"), ("asym_damped", "asym_forced")]

Then, we can run the analysis. To reduce the computing time, we can specify the bootstrap results obtained earlier (`df_bootstrap_single` parameter).

In [ ]:
df_bootstrap_compare = cosinor_nonlin.compare_pairs_n_comp_bootstrap_group(df, pairs, df_best_models=df_best_models, df_bootstrap_single=df_bootstrap, plot=True)#, folder="nonlin_gen_compare")

In [ ]:
df_bootstrap_compare[['test', 'n_components1', 'n_components2', 'd_amplitude', 'q(d_amplitude)', 'd_acrophase', 'q(d_acrophase)', 'd_amplification', 'q(d_amplification)', 'd_lin_comp', 'q(d_lin_comp)']].round(3)

If this argument is not specified (or if a certain measurement is missing), bootstrap is ran (again) on individual fits.

In [ ]:
df_bootstrap_compare2 = cosinor_nonlin.compare_pairs_n_comp_bootstrap_group(df, pairs, df_best_models=df_best_models, bootstrap_size=100, plot=True)

In [ ]:
df_bootstrap_compare2

Above, the parameters of the best models were identified using the `df_best_models` argument. If this is not specified `n_components` is used for all the fits.

In [ ]:
df_bootstrap_compare3 = cosinor_nonlin.compare_pairs_n_comp_bootstrap_group(df, pairs, n_components=3, period=24, bootstrap_size=100, plot=True)

In [ ]:
df_bootstrap_compare3